In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Dataframe
compute the maximum value associated to each sensor.

In [1]:
inputPath  = "./" 
outputPath = "./" 

In [3]:
# Create a DataFrame
readingDF = spark.read.load(inputPath,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "sensorId")\
.withColumnRenamed("_c1", "date")\
.withColumnRenamed("_c2", "PM10")

In [5]:
#readingDF.printSchema()
#readingDF.show()

In [6]:
# Define one group for each value of sensorId and compute the maximum value in each group
sensorsMaxValueDF = readingDF.groupBy("sensorId").agg({"PM10": "max"})

In [8]:
#sensorsMaxValueDF.printSchema()
#sensorsMaxValueDF.show()

In [ ]:
# Store the result in the output folder
sensorsMaxValueDF.write.csv(outputPath, header=False)

In [ ]:
spark.stop()

## RDD
count the number of critical values (pm10>50) for each sensor.

In [ ]:
sc = SparkContext()
inputPath  = "./" 
outputPath = "./" 
readingsRDD = sc.textFile(inputPath)

In [ ]:
# Apply a filter transformation to select only the lines with PM10>50
readingsHighValueRDD = readingsRDD.filter(lambda PM10Reading: float(PM10Reading.split(',')[2])>50 )

In [ ]:
# Create an RDD of key-value pairs
# Each pair contains a sensorId (key) and +1 (value)
# The function of the map transformation returns a tuple
sensorsPM10CriticalValuesRDD = readingsHighValueRDD.map(lambda PM10Reading: (PM10Reading.split(',')[0], 1) )

In [ ]:
# Count the number of critical values for each sensor by using the reduceByKey transformation.
sensorsCountsRDD = sensorsPM10CriticalValuesRDD.reduceByKey(lambda value1, value2: value1+value2)

## SQL
count the number of critical values (pm10>50) for each sensor and keep only the sensors which have more than one critical value.

In [ ]:
inputPath  = "./" 
outputPath = "./"

In [ ]:
# Create a DataFrame 
readingDF = spark.read.load(inputPath,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "sensorId")\
.withColumnRenamed("_c1", "date")\
.withColumnRenamed("_c2", "PM10")

In [ ]:
#readingDF.printSchema()
#readingDF.show()

In [ ]:
# "register" readingDF
readingDF.createOrReplaceTempView("readings")

In [ ]:
# Define one group for each value of sensorId and count the number of records with PM10>50.
# Select only the sensors with at least two high values
sensorsMaxValueDF = spark.sql("""SELECT sensorId, count(*)
FROM readings
WHERE PM10>50
GROUP BY sensorId
HAVING count(*)>=2""")

In [ ]:
#sensorsMaxValueDF.printSchema()
#sensorsMaxValueDF.show()

In [ ]:
# Store the result in the output folder
sensorsMaxValueDF.write.csv(outputPath, header=False)

In [ ]:
spark.stop()

## Obtain for each sensor the list of date with a critical value (pm10>50)

In [ ]:
sc = SparkContext()
inputPath  = "./" 
outputPath = "./"
readingsRDD = sc.textFile(inputPath)

In [ ]:
# Apply a filter transformation to select only the lines with PM10>50
readingsHighValueRDD = readingsRDD.filter(lambda PM10Reading: float(PM10Reading.split(',')[2])>50 )

In [ ]:
# Create an RDD of key-value pairs
# Each pair contains a sensorId (key) and a date (value)
sensorsCriticalDatesRDD = readingsHighValueRDD.map(lambda PM10Reading: (PM10Reading.split(',')[0], 
                                                                        PM10Reading.split(',')[1]) )

In [ ]:
# Create one pair for each sensor (key) with the list of dates associated with that sensor (value)
# by using the groupByKey transformation
finalSensorCriticalDates = sensorsCriticalDatesRDD.groupByKey()

In [ ]:
# The map method is used to transform the content of the iterable 
# over the values of each key into a list (that can be stored in a readable format)
finalSensorCriticalDateStringFormat = finalSensorCriticalDates.mapValues(lambda dates : list(dates))

In [ ]:
# Store the result in the output folder
finalSensorCriticalDateStringFormat.saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Obtain for each sensor the list of date with a critical value (pm10>50), including the sensor without critical values

In [ ]:
sc = SparkContext()
inputPath  = "./" 
outputPath = "./"
inputRDD = sc.textFile(inputPath)

In [ ]:
# Filter the lines with a pm10 value greater than 50
highPM10RDDValues = inputRDD.filter(lambda line : float(line.split(',')[-1]) > 50)\
                    .map(lambda line : (line.split(',')[0], line.split(',')[1]))

In [ ]:
# At this point we apply groupByKey() to concatenate the date in a list for the same key
highPM10RDDValuesGroupByKey = highPM10RDDValues.groupByKey()

In [ ]:
# I need to apply mapValues to transform the value part
highPM10RDDmapValues = highPM10RDDValuesGroupByKey.mapValues(lambda dates : list(dates))

In [ ]:
# I am missing the sensor which have no pm10 > 50, so I need to subtract from the input RDD the keys of the 
# RDD where I have the sensor with higher pm10 values
neverHighPM10RDD = inputRDD.map(lambda v1 : v1.split(',')[0]).subtract(highPM10RDDmapValues.keys())

In [ ]:
neverHighPM10RDD = neverHighPM10RDD.map(lambda v1 : (v1, []))
neverHighPM10RDD.collect()

In [ ]:
finalRDD = highPM10RDDmapValues.union(neverHighPM10RDD)

In [ ]:
finalRDD.saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Order sensors with the respect to the critical values in descending order

In [ ]:
sc = SparkContext()
inputPath  = "./" 
outputPath = "./"
inputRDD = sc.textFile(inputPath)

In [ ]:
# Filter rows which pm10 value is greater than 50
filteredRDD = inputRDD.filter(lambda line : float(line.split(',')[-1]) > 50)
filteredRDD.collect()

In [ ]:
# Emit key value pair (sid, +1)
sensorIdCounterRDD = filteredRDD.map(lambda s : (s.split(',')[0], 1))
sensorIdCounterRDD.collect()

In [ ]:
# I reduceByKey the previous RDD to aggregate the values
sensorIdCounterRDDAggr = sensorIdCounterRDD.reduceByKey(lambda v1,v2 : v1+v2)
sensorIdCounterRDDAggr.collect()

In [ ]:
# Now I want to order the RDD, I can apply sortBy
sortedRDD = sensorIdCounterRDDAggr.sortBy(lambda pair : pair[1], False)
sortedRDD.collect()

In [ ]:
sortedRDD.saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Find the sensor with the maximum value of pm10 associated

In [ ]:
sc = SparkContext()
inputPath  = "./" 
outputPath = "./"
inputRDD = sc.textFile(inputPath)

In [ ]:
highPM10RDD = inputRDD.filter(lambda line : float(line.split(',')[-1]) > 50)

In [ ]:
# I emit the key-value pairs (sId, +1)
keyValueSensorRDD = highPM10RDD.map(lambda line : (line.split(',')[0], 1) )

In [ ]:
# I reduceByKey the RDD
counterSensor = keyValueSensorRDD.reduceByKey(lambda v1,v2 : v1+v2).top(1, lambda pair : pair[1])
# This is a python variable
counterSensor

In [ ]:
# This is an RDD
counterSensorRDD = sc.parallelize(counterSensor, 1)

In [ ]:
counterSensorRDD.saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Associate to each question id the questions and the answers

In [ ]:
sc = SparkContext()
questions  = "./"
answers  = "./"
outputPath = "./" 

In [ ]:
questionsRDD = sc.textFile(questions)

In [ ]:
questionsRDD.collect()

In [ ]:
# I am interested in the questionid and the question, I apply a map function, 
# I split the file and I take the first and third piece
questionRDDFiltered = questionsRDD.map(lambda line : (line.split(',')[0], line.split(',')[2]) )
questionRDDFiltered.collect()

In [ ]:
answersRDD = sc.textFile(answers)
answersRDD.collect()

In [ ]:
# I need to extract the second and the last column from the answersRDD
answersRDDFiltered = answersRDD.map(lambda line : (line.split(',')[1], line.split(',')[3]) )
answersRDDFiltered.collect()

In [ ]:
finalRDDToMap = questionRDDFiltered.cogroup(answersRDDFiltered)
finalRDD = finalRDDToMap.mapValues(lambda value : (list(value[0]), list(value[1]))  )
finalRDD.collect()

In [ ]:
finalRDDToMap.collect()

In [ ]:
finalRDD.saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Detecting misleading users based on the watched movies and preferences expressed

In [ ]:
sc = SparkContext()
inputPath  = "./"
inputPath2  = "./"
inputPath3  = "./"
outputPath = "./" 
threshold = 0.5

In [ ]:
watchedMoviesRDD = sc.textFile(inputPath)
watchedMoviesRDD.collect()

In [ ]:
# I need only the pair (userId, Movie)
usersAndMovies = watchedMoviesRDD.map(lambda line : (line.split(",")[1], line.split(",")[0]) )
usersAndMovies.collect()

In [ ]:
moviesAndGenre = sc.textFile(inputPath2).map(lambda line:(line.split(",")[0], line.split(",")[2]))
moviesAndGenre.collect()

In [ ]:
# Now I need to join the 2 RDD
usersAndGenres = usersAndMovies.join(moviesAndGenre).map(lambda line : line[1])
usersAndGenres.collect()

In [ ]:
preferences = sc.textFile(inputPath3).map(lambda line : (line.split(",")[0], line.split(",")[1]))
preferences.collect()

In [ ]:
# Now I want to understand if the user watched films of genres who did not specified
# I can use the cogroup transforamtion to get for each users 2 lists
# the first is the list of genres he watched (the true)
# the second list is what ge declared (the possible fake)
usersWithGenresAndPrefer = usersAndGenres.cogroup(preferences)
usersWithGenresAndPrefer = usersWithGenresAndPrefer.mapValues(lambda value : (list(value[0]),list(value[1])))
usersWithGenresAndPrefer.collect()

In [ ]:
def misleadingUser(line):
    
    # I need to compare this two lists and compute the percentage
    # of genres watched not in the preference list
    watchedGenre = line[1][0]
    preferences = line[1][1]
    
    totalGenreWatched = 0
    totalGenreLiked = 0
    
    for movie in watchedGenre:
        totalGenreWatched += 1
        for pref in preferences:
            if(movie==pref):
                totalGenreLiked += 1
    
    
    if(((totalGenreWatched - totalGenreLiked)/totalGenreWatched) > threshold):
        return line

In [ ]:
misleadingUsers = usersWithGenresAndPrefer.filter(misleadingUser)
misleadingUsers = misleadingUsers.keys()

In [ ]:
misleadingUsers = misleadingUsers.coalesce(1)
misleadingUsers.saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Select window with an increasing trend (flatMap+groupByKey)

In [ ]:
sc = SparkContext()
inputPath  = "./" 
outputPath = "./"
inputRDD = sc.textFile(inputPath)
inputRDD.collect()

In [ ]:
# I want to export the following key value pair from the input RDD
# (timestamp of the record, [(timestamp, value), (timestamp-60, value), (timestamp-120, value)] )
def extractWindow(reading):
    timestamp = int(reading.split(",")[0])
    temperature = float(reading.split(",")[1])
    
    window = []
    
    window.append((timestamp, reading))
    window.append((timestamp-60, reading))
    window.append((timestamp-120, reading))
    
    return window

In [ ]:
windowTimestamp = inputRDD.flatMap(extractWindow)
windowTimestamp.collect()

In [ ]:
windowTimestampPerKey = windowTimestamp.groupByKey()
windowTimestampPerKey = windowTimestampPerKey.mapValues(lambda value : list(value))
windowTimestampPerKey.collect()

In [ ]:
# This function is used in the next transformation to select the windows with an incrasing temperature trend
def increasingTrendFunc(pairInitialTimestampWindow):

    # The key of the input pair is the intial timestamp of the current window
    minTimestamp = pairInitialTimestampWindow[0]
    
    # Store the (at most) 3 elements of the window in a dictionary
    # containing enties time stamp -> temperature
    timestampTemp = {}

    # pairInitialTimestampWindow[1] contains the elements of the current window
    window = pairInitialTimestampWindow[1]
    
    
    for timestampTemperature in window:
        fields = timestampTemperature.split(",")
        t = int(fields[0])
        temperature = float(fields[1])
        
        timestampTemp[t] = temperature
        
    
    # Check if the list contains three elements.
    # If the number of elements is not equal to 3 the window is incomplete and must be discarded
    if len(timestampTemp) != 3:
        increasing = False
    else:
        # Check is the increasing trend is satisfied
        if timestampTemp[minTimestamp]<timestampTemp[minTimestamp+60] and timestampTemp[minTimestamp+60]<timestampTemp[minTimestamp+120]:
            increasing = True
        else:
            increasing = False
            
    return increasing

In [ ]:
seletedWindowsRDD = windowTimestampPerKey.filter(increasingFunc)

In [ ]:
# The result is in the value part of the returned pairs
seletedWindowsRDD.values().map(lambda window: list(window)).collect()

In [ ]:
# Store the result. Map the iterable associated with each window to a list
seletedWindowsRDD.values().map(lambda window: list(window)).saveAsTextFile(outputPath)

In [ ]:
sc.stop()

## Dataframe (selectExpr): discretized age

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
inputPath = "./"
outputPath = "./"

In [ ]:
# Read the content of the input file profiles.csv and store it into a DataFrame
# The input file has an header
# Schema of the input data:
# |-- name: string (nullable = true)
# |-- surname: string (nullable = true)
# |-- age: integer (nullable = true)
profilesDF = spark.read.load(inputPath,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
profilesDF.show()
profilesDF.printSchema()

In [ ]:
# Define a User Defined Function called AgeCategory(age)
# that returns a string associated with the Category of the user.
# AgeCategory = "[(age/10)*10-(age/10)*10+9]"
# e.g.,
# 43 -> [40-49]
# 39 -> [30-39]
# 21 -> [20-29]
# 17 -> [10-19]
# ..
spark.udf.register("AgeCategory", lambda age: "["+str((age//10)*10)+"-"+str((age//10)*10+9)+"]")

In [ ]:
# Define a DataFrame with the following schema:
# |-- name: string (nullable = true)
# |-- surname: string (nullable = true)
# |-- rangeage: String (nullable = true)

profilesDiscretizedAge = profilesDF.selectExpr("name", "surname", "AgeCategory(age) as rangeage")

In [ ]:
profilesDiscretizedAge.printSchema()
profilesDiscretizedAge.show()

In [ ]:
# Save the result in the output folder
profilesDiscretizedAge.write.csv(outputPath, header=True)

In [ ]:
spark.stop()

## Dataframe SQL: discretized age

In [ ]:
inputPath = "./"
outputPath = "./"

In [ ]:
# Read the content of the input file profiles.csv and store it into a DataFrame
# The input file has an header
# Schema of the input data:
# |-- name: string (nullable = true)
# |-- surname: string (nullable = true)
# |-- age: integer (nullable = true)
profilesDF = spark.read.load(inputPath,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Define a User Defined Function called AgeCategory(age)
# that returns a string associated with the Category of the user.
# AgeCategory = "[(age/10)*10-(age/10)*10+9]"
# e.g.,
# 43 -> [40-49]
# 39 -> [30-39]
# 21 -> [20-29]
# 17 -> [10-19]
# ..
spark.udf.register("AgeCategory", lambda age: "["+str((age//10)*10)+"-"+str((age//10)*10+9)+"]")

In [ ]:
profilesDF.registerTempTable("profiles")

In [ ]:
# Define a DataFrame with the following schema:
# |-- name: string (nullable = true)
# |-- surname: string (nullable = true)
# |-- rangeage: String (nullable = true)
profilesDiscretizedAge = spark.sql("""SELECT name, surname, AgeCategory(age) as rangeage
FROM profiles""")

In [ ]:
#profilesDiscretizedAge.printSchema()
#profilesDiscretizedAge.show()

In [ ]:
# Save the result in the output folder
profilesDiscretizedAge.write.csv(outputPath, header=True)

In [ ]:
spark.stop()

## Dataframe (selectExpr): concatenation of fields

In [ ]:
inputPath = "./"
outputPath = "./"

In [ ]:
# Read the content of the input file profiles.csv and store it into a DataFrame
# The input file has an header
# Schema of the input data:
# |-- name: string (nullable = true)
# |-- surname: string (nullable = true)
# |-- age: integer (nullable = true)
profilesDF = spark.read.load(inputPath,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Define a User Defined Function called Concatenate(name, surname)
# that returns a string associated with the concatenation of name and surname.
spark.udf.register("Concatenate", lambda name, surname: name+" "+surname)

In [ ]:
# Define a DataFrame with the following schema:
# |-- name_surname: string (nullable = true)
namesDF = profilesDF.selectExpr("Concatenate(name, surname) as name_surname")

In [ ]:
namesDF.printSchema()
namesDF.show()

In [ ]:
# Save the result in the output folder
namesDF.write.csv(outputPath, header=True)

In [ ]:
spark.stop()

## Dataframe SQL: concatenation of fields

In [ ]:
inputPath = "./"
outputPath = "./"

In [ ]:
# Read the content of the input file profiles.csv and store it into a DataFrame
# The input file has an header
# Schema of the input data:
# |-- name: string (nullable = true)
# |-- surname: string (nullable = true)
# |-- age: integer (nullable = true)
profilesDF = spark.read.load(inputPath,\
                             format="csv",
                             header=True,\
                             inferSchema=True)

In [ ]:
# Define a User Defined Function called Concatenate(name, surname)
# that returns a string associated with the concatenation of name and surname.
spark.udf.register("Concatenate", lambda name, surname: name+" "+surname)

In [ ]:
profilesDF.registerTempTable("profiles")

In [ ]:
# Define a DataFrame with the following schema:
# |-- name_surname: string (nullable = true)
namesDF = spark.sql("SELECT Concatenate(name, surname) as name_surname FROM profiles")

In [ ]:
#namesDF.printSchema()
#namesDF.show()

In [ ]:
# Save the result in the output folder
namesDF.write.csv(outputPath, header=True)

In [ ]:
spark.stop()